# Instructions for downloading and installing anaconda

# TSCC
TSCC houses our 640-core supercomputer as part of a condo resource sharing system which allows other researchers (including bioinformaticians from other labs) to use our portion for their jobs (with an 8 hour time limit) which allows us to use their portions when we need extra computing power. In this course, we will be using TSCC to store and process our single-cell datasets.

# First steps

Your first login session should include some of the following commands, which will familiarize you with the cluster, teach you how to do some useful tasks on the queue, and help you set up a common directory structure shared by everyone in the lab. 

### First, login to TSCC:

```ssh YOUR_TSCC_USERNAME@tscc-login2.sdsc.edu```

TSCC has two login nodes, login1 and login2 for load-balancing (i.e. so if you just log on to tscc.sdsc.edu, it’ll choose whichever login node is less occupied. We’re logging in to a specific node because then we’ll always have our screen session on the same node) This is logging specifically on to login2. You can do login1 if you like, as well, to balance it out :)

### Download and install anaconda:

Download the Anaconda Python/R package manager using wget (web-get). The link below is from the Anaconda downloads page.

(Anaconda2 for python 2.7): ```wget https://repo.anaconda.com/archive/Anaconda2-5.2.0-Linux-x86_64.sh```

##### Note: You may also see references to "Anaconda3", which uses Python3 by default. We'll be using Anaconda2, which provides Python2, however we can still leverage conda to create environments with Python3. In other words, even though we'll be using Anaconda2, we will still be able to use both Python2 and Python3.

(Anaconda3 for python 3.6): ```https://repo.anaconda.com/archive/Anaconda3-5.2.0-Linux-x86_64.sh```

To install Anaconda2, run the shell script with bash (this will take some time). It will ask you a bunch of questions, but we'll be okay using the defaults for al (just press enter):

```bash Anaconda2-5.2.0-Linux-x86_64.sh```

To activate Anaconda2, source your .bashrc:

```source ~/.bashrc```

Make sure your Python is pointed to the Anaconda python with:

```which python```

The output should look something like:

```~/anaconda2/bin/python```

### Create a virtual environment on TSCC

Virtual environments are isolated programming environments that allow users to install packages specific to individual projects, without worrying about compatability among dependency issues. For example, if you were working on **Project A** which requires Package **V1.0**, but also working on **Project B** which requires Package **V2.0**, you could create two environments and work on both without worrying about any version conflicts.

There are a few ways to create virtual environments, but the easiest way for us to just use conda:

```bash
conda create --name $USER python=2.7 numpy pandas scipy scikit-learn matplotlib seaborn
```

which creates a conda environment using **$USER** (your username) as a name, and with the following packages installed:
- **python** (version 2.7)
- **numpy, pandas, scipy, scikit-learn, matplotlib, seaborn** (any versions that are compatible with each other)

### Creating virtual environments for packages required for this course
We will be using several packages each with its own set of dependencies, so let's create two more environments:

Copy/paste this code to create an environment called "Python2-single-cell". This (yada yada):
```bash
conda create -y -n Python2-single-cell 
source activate Python2-single-cell
conda install -y -c ...
```


Copy/paste this code to create an environment called "R-single-cell". This not only creates and activates our environment, but also uses 3 [channels](https://conda.io/docs/glossary.html#channels) (anaconda, r, conda-forge) to install some packages into our environment:

```bash
conda create -y -n R-single-cell 
source activate R-single-cell
conda install -y -c anaconda gcc=4.8.5 jupyter=1.0.0
conda install -y -c r r-base=3.4.1
conda install -y -c conda-forge libjpeg-turbo=1.5 zeromq=4.2.5
```

### Dependency hell
While conda does its best to manage packages/dependencies, there may be bugs. Unfortunately in this case, there is an error in the specific C library that is softlinked, as well as a path to an environment variable that needs to be linked. We'll need to fix this, and to do so, run through the steps very carefully:

Set some variables in shell
```bash
ENVNAME=R-single-cell
ANACONDAENV=/home/ucsd-trainXY/anaconda2/envs # replace 'ucsd-trainXY' with your username
```

Copy and run the following block of code, which:
- Changes directory to where the problem library is
- Moves the softlink somewhere else (just in case)
- Softlinks the appropriate library files (libstdc++.so.6.0.24 -> ./libstdc++.so, libstdc++.so.6.0.24 -> ./libstdc++.so.6)

```bash
cd ${ANACONDAENV}/${ENVNAME}/lib;

mv libstdc++.so ./libstdc++.so-bak;
mv libstdc++.so.6 ./libstdc++.so.6-bak;

ln -s libstdc++.so.6.0.24 ./libstdc++.so;
ln -s libstdc++.so.6.0.24 ./libstdc++.so.6;
```

### Installing Seurat inside R

The next step is to install the proper package dependencies required by Seurat. First, however, let's make sure we're using the correct R (that was installed from the r conda channel above):
```bash
which R
```

You should see:
```bash
~/anaconda2/envs/seurat/bin/R
```

Run R inside your terminal by typing: 
```bash
R
```

You should see:
```R
R version 3.4.1 (2017-06-30) -- "Single Candle"
Copyright (C) 2017 The R Foundation for Statistical Computing
Platform: x86_64-pc-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

  Natural language support but running in an English locale

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

> 
```

First, let's install devtools, which is required by Seurat and also lets us specify specific versions of packages to install:
```R
install.packages('devtools') # Pick http and a region that's close to us
```

Next, install stringi:
```R
install.packages("stringi", configure.args=c("--disable-cxx11"), repos="https://cran.rstudio.com")
```

After installing these packages, you should be able to run this entire block of code without any problems:
```R
devtools::install_version("uuid","0.1-2")
devtools::install_version("repr","0.15.0")
devtools::install_version("IRdisplay","0.5.0")
devtools::install_version("evaluate","0.10.1")
devtools::install_version("crayon","1.3.4")
devtools::install_version("digest","0.6.15")
devtools::install_version("curl","3.2")
devtools::install_version("lubridate","1.7.4")
devtools::install_version("Rcpp","0.12.17")
devtools::install_version("httpuv", "1.3.5") # current version breaks
devtools::install_version("shiny", "1.0.5") # current shiny requires httpuv 1.4.3
devtools::install_version("pillar", "1.2.1") # current pull looks for nonexist 1.2.2
devtools::install_version("CVST", "0.2") # current pull looks for nonexist
devtools::install_version("DRR", "0.0.3")
devtools::install_version("dimRed", "0.1.0")
devtools::install_version("Hmisc", "4.1-1")
devtools::install_version("htmlTable", "1.11.1") # current pull references nonexist
devtools::install_version("recipes", "0.1.2")
devtools::install_version("tibble", "1.4.2")
devtools::install_version("ggplot2", "2.2.1")
devtools::install_version("crosstalk", "1.0.0")
devtools::install_version("plotly", "4.7.1")
devtools::install_version("caret", "6.0-78") # current pull references nonexist
devtools::install_version("Seurat","2.3.1")
devtools::install_version("pbdZMQ","0.3-3")
```

### Installing DESeq2 inside R
Sometimes, we can leverage a package manager called bioconductor to help us install packages (we would have done this for Seurat if Seurat were part of bioconductor). Fortunately, DESeq2 can be installed in this manner:

Run this inside R
```R
source("https://bioconductor.org/biocLite.R")
biocLite("DESeq2")  # upgrade all should be fine, otherwise upgrade none
```

### Making Seurat and DESeq available in your jupyter notebook
We can load and use these packages inside the terminal as-is, however it's nice to be able to view things like plots and stuff inside Jupyter. Let's run the appropriate commands inside R, which installs IRKernel, and registers our environment (single-cell) to our Jupyter installation:

```R
options(unzip = 'internal')
devtools::install_github('IRkernel/IRkernel')
IRkernel::installspec(name = 'single-cell', displayname = 'single-cell')
```